# Video Cropping

Contains 2 sections
1. Exploration - where interactively you can pick the correct dimensions for cropping.
2. Cropping the video according to the configured dimensions.

In [39]:
import subprocess

**Declare variables**
- `input_video` = source video file location
- `ouptut_video` = output location for cropped video
-  `crop_dimension` = width:height dimension for cropped video. It will be center cropped. Use the `input_video_size_info` command to get info about the width / height size of the `input_video`

In [40]:
# specify input video, and output path
input_video = "./data/RWPro_input.mp4"
ouptut_video = "./data/RWPro_final.mp4"

# displays width and height dimensions of the input video
input_video_size_info = [
    'ffprobe', '-v', 'error', '-select_streams', 'v:0',
    '-show_entries', 'stream=width,height', input_video
]
try:
    subprocess.run(input_video_size_info, check=True)
except:
    pass

[STREAM]
width=4096
height=1212
[/STREAM]


# #1. Exploration

### Explore video crop dimensions
This will start a player with video cropped to the specified dimemsion.

`crop_dimension` format is `WIDTH:HEIGHT`. 

Tip: keep height same as orignal video and crop width for the general problem of wide video area.

In [ ]:
# specify the dimensions to crop to. WIDTH:HEIGHT
crop_dimension = "2000:1212"

explore_crop_dimensions = [
    'ffplay', '-i', input_video,
    '-vf', f'crop={crop_dimension}'
]
try:
    subprocess.run(explore_crop_dimensions, check=True)
except subprocess.CalledProcessError as e:
    pass

### Explore speaker head dimensions
Starts a player where you can test the correct dimensions for the speaker's head.

In [ ]:
# specify dimensions for speaker's head. Format: WIDTH:HEIGHT:X:Y
# X, Y in graphics coordinates: 0.0 point is top, left.
speaker_head_dimensions = "320:180:3776:0"

explore_speaker_coordinates = [
    'ffplay', '-i', input_video,
    '-vf', f'crop={speaker_head_dimensions}'
]
try:
    subprocess.run(explore_speaker_coordinates, check=True)
except subprocess.CalledProcessError as e:
    pass

# #2. Create final video. Merge cropped video with speaker's head video.

Switch `testing` to `True` to only process just 5 minutes of video.

Overlay formats:
- `overlay=W-w:0`: top right corner.
- `overlay=W-w-10:H-h-10`: bototm right corner with 10px spacing.

`W`, `H` - represents width / height of first caption (`cropped`)

`w`, `h`- represents width / height of second caption (`face`)

In [41]:
# testing = True - means only process first 5 minutes of video.
testing = False

merge_videos_command = [
    'ffmpeg', '-y', '-i', input_video, '-filter_complex',
    f'[0:v]crop={crop_dimension}[cropped], [0:v]crop={speaker_head_dimensions}[face]; [cropped][face] overlay=W-w:0',
    '-c:a', 'copy', ouptut_video
    ]

if testing:
    merge_videos_command.insert(-3, '-t')
    merge_videos_command.insert(-3, '300')

# print(" ".join(merge_videos_command))

try:
    subprocess.run(merge_videos_command, check=True)
except subprocess.CalledProcessError as e:
    pass

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.3)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --ena

---
### [Alternative] If you only want to crop the main video, without including the speaker head.

In [ ]:

crop_video_command = [
    'ffmpeg', '-i', input_video,
    '-filter:v', f'crop={crop_dimension}',
    '-c:a', 'copy', ouptut_video
]
try:
    subprocess.run(crop_video_command, check=True)
except subprocess.CalledProcessError as e:
    pass